In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

In [ ]:
ke_data=pd.read_csv('ke.csv')
ke_data.head()

In [ ]:
# Creating the geopandas dataframe
kenya=gpd.GeoDataFrame(ke_data,geometry=gpd.points_from_xy(ke_data['lng'],ke_data['lat']))

In [ ]:
# set the CRS to {'init':4326}
kenya.crs={'init':'epsg:4326'}


### Which cities are located above the equator?Below?

In [ ]:
kenya[kenya.geometry.y>0][['city','lat']]

#### Furthesr Nort,South,east,West

In [ ]:
north = kenya.loc[kenya.geometry.y.idxmax()]
south = kenya.loc[kenya.geometry.y.idxmin()]
east = kenya.loc[kenya.geometry.x.idxmax()]
west = kenya.loc[kenya.geometry.x.idxmin()]

north['city'], south['city'], east['city'], west['city']


#### Closest City to Nairobi

In [ ]:
from shapely.ops import nearest_points

# Get Nairobi point
nairobi = kenya[kenya.city == "Nairobi"].geometry.values[0]

# Compute distances
kenya['distance_to_nairobi_km'] = kenya.distance(nairobi) * 111  # rough km estimate

# Closest (excluding Nairobi itself)
kenya[kenya.city != "Nairobi"].sort_values('distance_to_nairobi_km').head(1)


## Cities over 1 million

In [ ]:
kenya[kenya.population > 1_000_000][['city', 'population']]


## Cities within 100 km of Mombasa

In [ ]:
from geopy.distance import geodesic

# Get Mombasa's coordinates
mombasa_coords = kenya[kenya.city == "Mombasa"].geometry.values[0].coords[0]

# Compute geodesic distance for each city
kenya['dist_to_mombasa_km'] = kenya.geometry.apply(
    lambda x: geodesic((x.y, x.x), (mombasa_coords[1], mombasa_coords[0])).km
)

kenya[kenya['dist_to_mombasa_km'] <= 100][['city', 'dist_to_mombasa_km']]


## Most Isolated City

In [ ]:
from scipy.spatial import distance_matrix

# Coordinates matrix
coords = list(zip(kenya.geometry.y, kenya.geometry.x))
dist_matrix = distance_matrix(coords, coords)

# Sum of distances to others
kenya['isolation_score'] = dist_matrix.sum(axis=1)

# Most isolated city
kenya.loc[kenya['isolation_score'].idxmax()][['city', 'isolation_score']]


## Buffer each city and check overlaps

In [ ]:
kenya_buffered = kenya.to_crs(epsg=3857).copy()  # project for distance-based buffering
kenya_buffered['buffer_50km'] = kenya_buffered.buffer(50_000)

# Example: Check overlap with other buffers for Nairobi
nairobi_buffer = kenya_buffered[kenya_buffered.city == "Nairobi"].buffer_50km.values[0]

kenya_buffered[kenya_buffered.buffer_50km.intersects(nairobi_buffer)][['city']]


## Plot cities on Map

In [ ]:
import folium

# Center the map around Kenya
m = folium.Map(location=[0.5, 37], zoom_start=6)

# Add all cities
for _, row in kenya.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        popup=f"{row['city']} ({int(row['population']) if pd.notnull(row['population']) else 'N/A'})",
        color='blue',
        fill=True,
    ).add_to(m)

m


## Mean Population

In [ ]:
kenya['population'].mean()


## Population-Weighted Center(Centroid)

In [ ]:
import numpy as np

kenya['weight'] = kenya['population'].fillna(0)
x = np.average(kenya.geometry.x, weights=kenya['weight'])
y = np.average(kenya.geometry.y, weights=kenya['weight'])
(f"Population-weighted centroid: lat={y:.4f}, lon={x:.4f}")


In [ ]:
import folium

# Center map around Kenya's centroid
m = folium.Map(location=[kenya.geometry.y.mean(), kenya.geometry.x.mean()], zoom_start=6)

# Add cities as circle markers
for _, row in kenya.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        popup=row['city'],
        color='blue',
        fill=True,
        fill_opacity=0.6
    ).add_to(m)

m


In [ ]:
pip install leafmap

In [ ]:
import leafmap

# Create interactive map
m = leafmap.Map(center=[0.3, 37], zoom=6)

# Loop through cities and add satellite basemap tiles
for _, row in kenya.iterrows():
    m.add_marker(
        location=(row.geometry.y, row.geometry.x),
        popup=row['city'],
        icon='map-marker'
    )

# Set map to satellite view
m.add_basemap("Esri.WorldImagery")
m
